In [3]:
import dgl
import time
import torch as th
import numpy as np
import torch.nn.functional as F

from load_data import load_ogb

The code here shows the baseline of using MLP to classify nodes with the spectral embeddings of the nodes. Even though the spectral embeddings contain the global information of the graph. It seems a simple linear decomposition cannot achieve very good performance.

In [4]:
class Dataset(th.utils.data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, list_IDs, labels,features):
        'Initialization'
        self.labels = labels
        self.list_IDs = list_IDs
        self.features=features

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)

  def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]

        # Load data and get label
        X =self.features[ID]
        y = self.labels[ID]

        return X, y

class ClassifierMLP(th.nn.Module):
    def __init__(self, input_size, hidden_size,out_size,single_layer=False):
        super(ClassifierMLP, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.out_size=out_size
        self.single_layer=single_layer
        if self.single_layer:
            self.fc1 = th.nn.Linear(self.input_size, self.out_size)
        else:
            self.fc1 = th.nn.Linear(self.input_size, self.hidden_size)
            self.relu = th.nn.ReLU()
            self.fc2 = th.nn.Linear(self.hidden_size, self.out_size)
        self.activation=partial(F.softmax, dim=1)

    def forward(self, x):
        if self.single_layer:
            output=self.fc1(x)
        else:
            hidden = self.fc1(x)
            relu = self.relu(hidden)
            output = self.fc2(relu)
        return output

In [5]:
from functools import partial

def compute_acc(results, labels, multilabel):
    """
    Compute the accuracy of prediction given the labels.
    """
    if multilabel:
        predicted_labels = th.sigmoid(results).data > 0.5
        return th.sum(predicted_labels.cpu() == labels.cpu()).item() / labels.numel()
    else:
        labels = labels.long()
        return (results == labels).float().sum() / len(results)

def evaluate_mlp_model(model,data_loader,multilabel,device):
    labels=None
    for local_batch, local_labels in data_loader:
        pred = model(local_batch)
        local_labels = local_labels.to(device)

        if multilabel is False:
            pred = pred.argmax(dim=1)
        else:
            pred = pred
        if labels is None:
            labels=local_labels
            logits=pred
        else:
            labels=th.cat((labels,local_labels))
            logits = th.cat((logits, pred))

    acc=compute_acc(results=logits, labels=labels, multilabel=multilabel)
    return acc


def compute_acc_MLP_pytorch(feats, labels, train_idx, val_idx, test_idx, num_classes, device,
                            n_hidden=100, lr_d=1e-4, n_cepochs=50,
                            l2norm = 0.0001, batch_size=1000, multilabel=False):

    ###
    # Use the encoded features for classification
    # Here we initialize the features using the reconstructed ones

    # feats = g.nodes[category].data['features']

    inp_dim = feats.shape[1]
    model = ClassifierMLP(input_size=inp_dim, hidden_size=n_hidden,out_size=num_classes)
    model = model.to(device)
    feats=feats[np.arange(labels.shape[0])].to(device)

    # optimizer
    optimizer = th.optim.Adam(model.parameters(), lr=lr_d, weight_decay=l2norm)

    # training loop
    print("start training...")
    forward_time = []
    backward_time = []
    model.train()

    # TODO find all zero indices rows and remove.
    if len(labels.shape)>1:
        zero_rows=np.where(~(labels).cpu().numpy().any(axis=1))[0]
        print("Len of zero rows "+str(len(zero_rows)))

        train_idx=np.array(list(set(train_idx).difference(set(zero_rows))))
        val_idx = np.array(list(set(val_idx).difference(set(zero_rows))))
        test_idx = np.array(list(set(test_idx).difference(set(zero_rows))))

    train_indices = th.tensor(train_idx).to(device).long()
    valid_indices = th.tensor(val_idx).to(device).long()
    test_indices = th.tensor(test_idx).to(device).long()
    ## Minibatch training parameters
    params = {'batch_size': batch_size,
              'shuffle': True,
              'num_workers': 0}
    training_set = Dataset(train_idx, labels,feats)
    training_generator = th.utils.data.DataLoader(training_set, **params)

    validation_set = Dataset(val_idx, labels,feats)
    validation_generator = th.utils.data.DataLoader(validation_set, **params)

    test_set = Dataset(test_idx, labels,feats)
    test_generator = th.utils.data.DataLoader(test_set, **params)


    best_val_acc = 0
    best_test_acc = 0
    if multilabel is False:
        loss_func = th.nn.CrossEntropyLoss()
    else:
        loss_func = th.nn.BCEWithLogitsLoss()
    for epoch in range(n_cepochs):
        for local_batch, local_labels in training_generator:
            optimizer.zero_grad()
            pred = model(local_batch)
            local_labels = local_labels.to(device)
            loss = loss_func(pred, (local_labels))


            loss.backward()
            optimizer.step()
            if multilabel is False:
                pred = pred.argmax(dim=1)
            else:
                pred = pred
            print_debug=False
            if print_debug:
                train_auc = compute_acc(results=pred, labels=local_labels, multilabel=multilabel)
                print('Step loss %.4f, Train Acc %.4f' % (
                         loss.item(),
                         train_auc.item() if th.is_tensor(train_auc) else train_auc
                     ))
        if epoch % 5 == 0:
            val_acc = evaluate_mlp_model(model,validation_generator,multilabel,device)
            test_acc = evaluate_mlp_model(model,test_generator,multilabel,device)

            if best_val_acc < val_acc:
                best_val_acc = val_acc
                best_test_acc = test_acc
            print('Epoch %.1f, Loss %.4f, Val Acc %.4f (Best %.4f), Test Acc %.4f (Best %.4f)' % (
                epoch,
                loss.item(),
                #train_auc.item() if th.is_tensor(train_auc) else train_auc,
                val_acc.item()if th.is_tensor(val_acc) else val_acc,
                best_val_acc.item()if th.is_tensor(best_val_acc) else best_val_acc,
                test_acc.item()if th.is_tensor(test_acc) else test_acc,
                best_test_acc.item()if th.is_tensor(best_test_acc) else best_test_acc
            ))
    return best_val_acc,best_test_acc

In [7]:
g, num_labels = load_ogb('ogbn-products')
g.ndata.pop('features')

in_feats = 128
eigen_vals, eigen_vecs = get_eigen(g, in_feats, 'ogbn-products')
nfeat = th.tensor(eigen_vecs * np.sqrt(eigen_vals).reshape((1, len(eigen_vals))),
                  dtype=th.float32)
labels = g.ndata.pop('labels')

hyperparams = {
    'batch_size': 1024,
    'num_workers': 4,
    'num_hidden': 256,
    'num_layers': 3,
    'dropout': 0.5,
    'lr': 0.003,
    'num_epochs': 20,
    'fanouts': [15,10,5],
    'eval_batch_size': 10000
}
device = th.device('cuda:7')

# Define model
train_nid = th.nonzero(g.ndata['train_mask'], as_tuple=True)[0]
val_nid = th.nonzero(g.ndata['val_mask'], as_tuple=True)[0]
test_nid = th.nonzero(~(g.ndata['train_mask'] | g.ndata['val_mask']), as_tuple=True)[0]
best_val_acc,best_test_acc = compute_acc_MLP_pytorch(nfeat, labels, train_nid, val_nid, test_nid, num_labels, device,
                                                     n_hidden=200, lr_d=1e-3, n_cepochs=100,
                                                     l2norm = 0.0001, batch_size=1000, multilabel=False)

load ogbn-products
finish loading ogbn-products
finish constructing ogbn-products
start training...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:68: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:69: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:70: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Epoch 0.0, Loss 2.1725, Val Acc 0.3169 (Best 0.3169), Test Acc 0.2702 (Best 0.2702)
Epoch 5.0, Loss 1.4487, Val Acc 0.5242 (Best 0.5242), Test Acc 0.3461 (Best 0.3461)
Epoch 10.0, Loss 1.2794, Val Acc 0.5581 (Best 0.5581), Test Acc 0.3627 (Best 0.3627)
Epoch 15.0, Loss 1.3839, Val Acc 0.5828 (Best 0.5828), Test Acc 0.3755 (Best 0.3755)
Epoch 20.0, Loss 1.3230, Val Acc 0.5927 (Best 0.5927), Test Acc 0.3811 (Best 0.3811)
Epoch 25.0, Loss 1.3238, Val Acc 0.5972 (Best 0.5972), Test Acc 0.3834 (Best 0.3834)
Epoch 30.0, Loss 1.2007, Val Acc 0.6049 (Best 0.6049), Test Acc 0.3876 (Best 0.3876)
Epoch 35.0, Loss 1.2295, Val Acc 0.6136 (Best 0.6136), Test Acc 0.3928 (Best 0.3928)
Epoch 40.0, Loss 1.1121, Val Acc 0.6109 (Best 0.6136), Test Acc 0.3910 (Best 0.3928)
Epoch 45.0, Loss 1.2097, Val Acc 0.6082 (Best 0.6136), Test Acc 0.3881 (Best 0.3928)
Epoch 50.0, Loss 1.1723, Val Acc 0.6151 (Best 0.6151), Test Acc 0.3929 (Best 0.3929)
Epoch 55.0, Loss 1.1863, Val Acc 0.6191 (Best 0.6191), Test Acc 0.3